# Aerodynamic rotor design
This notebook shows how to use the `lacbox.rotor_design.single_point_design` method to generated an aerodynamic wind turbine blade design for a single point operational conditions (wind-speed, rotor-speed, pitch, etc.). 

The method takes a set of geometrical and operational inputs and returns the blade planform (chord, twist, relative-thickness) as well as performance metrics (power-coefficient, thrust-coefficient, etc.).  

In [ ]:
# Loading modules
import matplotlib.pyplot as plt
import numpy as np
from lacbox.rotor_design import get_design_functions_1MW, single_point_design

Below the documentation for `lacbox.rotor_design.single_point_design` method is shown.

In [ ]:
help(single_point_design)

## *Example*: 1MW turbine
Below an example of input is given for a 1 MW turbine with a rotor radius of 35 m.

Below all the scalar inputs are set.

In [ ]:
R = 35  # Rotor radius [m]
tsr = 9.0  # Tip-Speed-Ratio [-]
r_hub = 1.0  # Hub radius [m]
chord_max = 3.0  # Maximum chord size [m]
chord_root = 2.7  # Chord size at the root [m]
B = 3  # Number of blades [#]

### Rotor-span
Below the **rotor-span** is set. This is *not* the *blade-span* (which for a value of zero would mean the hub to the blade tip) but the *rotor-span* which for a value of zero would be at the center of rotation until the blade tip. 

For the case below the rotor-span is starting at the rotor-hub (`r_hub`) and then it goes to the rotor-radius but making the last point slightly smaller than the rotor-radius (`R*(1-1e-5)`) to avoid issues with the tip-loss factor that yields numerical issues in this limit. The rotor-span is then divided into `40` point between these limits with equidistant spacing between point and including the limit.

In [ ]:
r = np.linspace(r_hub, R*(1-1e-3), 40)  # Rotor span [m]

### Blade absolute thickness
Below the **blade absolute thickness** input is defined. This the blade thickness in meters and it should be defined for each of the rotor-span location.

For the example below the thickness is defined from a polynomial (which defines a function for `t(r)`) but it also includes a minimum cap via the parameter `chord_root` to limit the thickness in the root part of the blade.

In [ ]:
def thickness(r, chord_root):
    """Absolute thickness [m] as a function of blade span [m] for 35-m blade"""
    p_edge = [
        9.35996e-8,
        -1.2911e-5,
        7.15038e-4,
        -2.03735e-2,
        3.17726e-1,
        -2.65357,
        10.2616,
    ]  # polynomial coefficients
    t_poly = np.polyval(p_edge, r)  # evaluate polynomial
    t = np.minimum(t_poly, chord_root)  # clip at max thickness
    return t

t = thickness(r, chord_root)  # Absolute thickness [m]

In [ ]:
# Plotting the absolute thickness
fig, ax = plt.subplots()

ax.plot(r, t)

ax.set_xlabel("Rotor-span [m]")
ax.set_ylabel("Blade thickness [m]")

In the case that one is starting from a known HAWC2 design the *blade absolute thickness* can be found from the [HAWC2 *AE-file*](https://lac-course.pages.windenergy.dtu.dk/dtulac/load_save_ae.html). 

This can be done like so (assuming that you loaded the file with the `lacbox.io.load_ae` function):
```python
r, c, tc, pcset = load_ae(*your-ae-filename*, unpack=True)
t = tc*c/100
```

Where the `/100` is because `tc` is in percent.

### Lift, Drag and Angle-of-Attack function
The method requires input for the lift-, drag-coefficient and Angle-of-Attack, but not as a value along the rotor-span or as function of Angle-of-Attack as it is in the *HAWC2 PC-file*. It should be a function, but the values should be a function of the blade thickness-to-chord ratio (referred to as `tc`). 

Below, such function are loading for the 1MW example, which also provides the points that was used to build the interpolation function. Here `*_des` are the design function (e.g. $C_l(t/c)$,$C_d(t/c)$, $\alpha(t/c)$) and `*_vals` and the values that was used to construct the interpolation functions (e.g. `tc_vals` vs. `cl_vals`, `tc_vals` vs. `cd_vals`, `tc_vals` vs. `aoa_vals`)

In [ ]:
cl_des, cd_des, aoa_des, tc_vals, cl_vals, cd_vals, aoa_vals = get_design_functions_1MW()

In [ ]:
# Plotting the functions and the constructing data
tc_plot = np.linspace(0, 100, 100)
fig1, axs1 = plt.subplots(3, 1, num=1)

axs1[0].plot(tc_plot, cl_des(tc_plot), "k", label="cl_des")
axs1[0].plot(tc_vals, cl_vals, "ok", label="cl_vals")
axs1[0].set_ylabel("$C_l$ [-]")
axs1[0].set_xlim(0, 100)
axs1[0].legend()

axs1[1].plot(tc_plot, cd_des(tc_plot), "k", label="cd_des")
axs1[1].plot(tc_vals, cd_vals, "ok", label="cd_vals")
axs1[1].set_ylabel("$C_d$ [-]")
axs1[1].set_xlim(0, 100)
axs1[1].legend()

axs1[2].plot(tc_plot, aoa_des(tc_plot), "k", label="aoa_des")
axs1[2].plot(tc_vals, aoa_vals, "ok", label="aoa_vals")
axs1[2].set_ylabel(r"$\alpha$ [-]")
axs1[2].set_xlabel(r"$t/c$ [deg]")
axs1[2].set_xlim(0, 100)
axs1[2].legend()

fig1.tight_layout()

For the `single_point_deign` function to work well it is important that $C_l(t/c)$ is monotonically decrease for increasing `tc` - otherwise the solution will end up with jumps in the resulting chord.

### Running the single-point-design method
Below the `single_point_design` method is run with the input defined above

In [ ]:
blade_design = single_point_design(
    r, t, tsr, R, cl_des, cd_des, aoa_des, chord_root, chord_max, B
)

### Plotting the output

In [ ]:
fig2, axs2 = plt.subplots(3, 1, figsize=(6.5, 6))

# Chord
axs2[0].plot(r, blade_design["chord"])
axs2[0].set_ylabel("Chord [m]")
axs2[0].set_xlim(0, R)
axs2[0].set_ylim(0, 3.4)

# Twist
axs2[1].plot(r, blade_design["twist"])
axs2[1].set_ylabel("Twist [deg]")
axs2[1].set_xlim(0, R)

# t/c
axs2[2].plot(r, blade_design["tc"])
axs2[2].set_ylabel("Rel. thickness [%]")
axs2[2].set_xlabel("Rotor span [m]")
axs2[2].set_xlim(0, R)
axs2[2].set_ylim(0, 102)

fig2.tight_layout()

In [ ]:
fig3, axs3 = plt.subplots(2, 2, num=3, clear=True)

# t/c
axs3[0, 0].plot(r, blade_design["tc"])
axs3[0, 0].set_ylabel("t/c [%]")
axs3[0, 0].set_xlim(0, R)
axs3[0, 0].set_ylim(0, 102)

# aoa
axs3[0, 1].plot(r, blade_design["aoa"])
axs3[0, 1].set_ylabel(r"$\alpha$ [deg]")
axs3[0, 1].set_xlim(0, R)
axs3[0, 1].yaxis.tick_right()
axs3[0, 1].yaxis.set_label_position("right")

# cl
axs3[1, 0].plot(r, blade_design["cl"])
axs3[1, 0].set_ylabel("$C_l$ [-]")
axs3[1, 0].set_xlabel("Span [m]")
axs3[1, 0].set_xlim(0, R)

# cd
axs3[1, 1].plot(r, blade_design["cd"])
axs3[1, 1].set_ylabel("$C_d$ [-]")
axs3[1, 1].set_xlabel("Span [m]")
axs3[1, 1].set_xlim(0, R)
axs3[1, 1].yaxis.tick_right()
axs3[1, 1].yaxis.set_label_position("right")

fig3.tight_layout()

In [ ]:
fig4, axs4 = plt.subplots(3, 1, num=4, clear=True, figsize=(6.5, 5.5))

# Local-Thrust-Coefficient
axs4[0].plot(r, blade_design["CLT"])
axs4[0].axhline(y=8 / 9, ls="--", color="k", lw=1)
axs4[0].set_ylabel("Local thrust ($C_{LT}$) [-]")
axs4[0].set_ylim(0, 1.0)
axs4[0].set_xlim(0, R)

# Local-Power-Coefficient
axs4[1].plot(r, blade_design["CLP"])
axs4[1].axhline(y=16 / 27, ls="--", color="k", lw=1)
axs4[1].set_ylabel("Local Power ($C_{LP}$) [-]")
axs4[1].set_xlim(0, R)
axs4[1].set_ylim(-0.4, 0.6)

# Axial Induction
axs4[2].plot(r, blade_design["a"])
axs4[2].axhline(y=1 / 3, ls="--", color="k", lw=1)
axs4[2].set_ylabel("Axial induction ($a$) [-]")
axs4[2].set_xlabel("Rotor span [m]")
axs4[2].set_xlim(0, R)

fig4.suptitle(f"$C_T$={blade_design['CT']:1.3f}, $C_P$={blade_design['CP']:1.3f}")
fig4.tight_layout()